In [1]:
# ESTUDO DO PROBLEMA DE ESCALONAMENTO DE HORÁRIOS E ALGORITMOS GENÉTICOS

# Cada programa terá vários cursos
# Cada programa terá n semestres
# Cada curso pertencerá à um semestre especifico
# Cada semestre não poderá superar mais do que 10 aulas semanais no horario

# AG       -> Programa{ Cursos, Profs, Salas } : Schedule
# Programa -> {Horario Semanal para cada o programa com n semestres}

# *OBS1: Haverá "ataques ou choque de horarios" em um schedule para programa com n semestres (ataque entre semestres)
# *OBS2: Após a geração de um schedule pra um programa, guardar restrições desse programa para aplicar em outro, caso use os mesmos requisitos (salas e professores)


# ------------ AG ------------
# ___Code___
# Laço:
# - Popular
# - Fitness
# - Testar -> (Sol, S/ Sol, Evoluir)
# Fim

# Evoluir:
# - Seleção
# - Cruzamento
# - Mutação
# __________

# Conceitos:
# • Definir o que é cromossomo
# • Teste de Meta: Sim, se algum cromossomo tem fitness = 1. Não, caso o contrário.
# • Fitness: (1 - Erro/MaxErro)
#                  (Acerto/MaxAcerto)
# • Seleção: Roleta Viciada (Gera 2*n casais de cromossomos. Partição da roleta é baseada no valor Fitness)
# • Cruzamento: Cruzar casais gerados na Seleção. Ao cruzar, é pegada informação do macho e da fêmea, e são gerados 2 filhos melhores geneticamente.
# • Mutação: Mutar gene de cromossomos aleatoriamente

In [2]:
#Import .ipynb as module: %run name.ipynb
import math
import random
import copy
import itertools as it

In [3]:
class Schedule:
    #     10 Aulas semanais:
    # | Seg  Ter  Qua  Qui  Sex |
    # |  1    3    5    7    9  |
    # |  2    4    6    8   10  |
    #
    def __init__(self):
        self.weekName = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex']
        self.week = [[0]*2]*5
    
    def __repr__(self):
        strOut = '| '
        for i in self.weekName:
            strOut += i + '  '
        strOut += ' |\n| '
        for i in self.week:
            strOut += ' ' + str(i[0]) + '   '
        strOut += ' |\n| '
        for i in self.week:
            strOut += ' ' + str(i[1]) + '   '
        strOut += ' |\n '
        return strOut
    
Schedule()

| Seg  Ter  Qua  Qui  Sex   |
|  0    0    0    0    0    |
|  0    0    0    0    0    |
 

In [4]:
### Estruturas de Dados para o problema ###

# Class Curso
class Curso:
    _id = 0
    def __init__(self, nome, aulas, semestre, profId=-1):
        Curso._id     = Curso._id +1
        self.id       = Curso._id
        self.nome     = nome
        self.aulas    = aulas
        self.semestre = semestre
        self.profId   = profId
        
    def __repr__(self):
        return 'Curso {}: {} ({} aulas, {}o semestre, {} Prof Id)'.format(self.id, self.nome, self.aulas, self.semestre, self.profId)

# Class Prof
class Prof:
    _id = 0
    def __init__(self, nome):
        Prof._id   = Prof._id +1
        self.id    = Prof._id
        self.nome  = nome
        
    def __repr__(self):
        return 'Prof {}: {}'.format(self.id, self.nome)
    
# Class Sala
class Sala:
    _id = 0
    def __init__(self, nome):
        Sala._id   = Sala._id +1
        self.id    = Sala._id
        self.nome  = nome
    
    def __repr__(self):
        return 'Sala {}: {}'.format(self.id, self.nome)

# Class Programa
class Programa:
    def __init__(self, nome, semestres, cursos=[], salas=[], profs=[]):
        self.nome      = nome
        self.semestres = semestres
        self.cursos    = cursos
        self.salas     = salas
        self.profs     = profs
        
    def addCurso(self, curso):
        try:
            if curso not in self.cursos:
                self.cursos.append(curso)
        except:
            return
    
    def removeCurso(self, curso):
        try:
            self.cursos.remove(curso)
        except:
            return
    
    def addProf(self, prof):
        try:
            if prof not in self.profs:
                self.profs.append(prof)
        except:
            return
    
    def removeProf(self, prof):
        try:
            self.profs.remove(prof)
        except:
            return
        
    def addSala(self, sala):
        try:
            if sala not in self.salas:
                self.salas.append(sala)
        except:
            return
    
    def removeSala(self, sala):
        try:
            self.salas.remove(sala)
        except:
            return
        
    def __repr__(self):
        return 'Programa: {} ({} semestres, {} cursos, {} salas, {} professores)'.format(self.nome, self.semestres, len(self.cursos), len(self.salas), len(self.profs))

#Testes
c1 = Curso('Calculo I', 2, 1)
c2 = Curso('Calculo II', 2, 2)
p1 = Prof('Enio Romagnome')
p2 = Prof('Roberto Carlos')
s1 = Sala('BC-23')
s2 = Sala('BC-08')
c1.profId = p1.id
c2.profId = p2.id

p1 = Programa('EngComp', 10, [c1, c2], [s1, s2], [p1, p2])

print(p1)
print(p1.cursos)
print(p1.profs)
print(p1.salas)

Programa: EngComp (10 semestres, 2 cursos, 2 salas, 2 professores)
[Curso 1: Calculo I (2 aulas, 1o semestre, 1 Prof Id), Curso 2: Calculo II (2 aulas, 2o semestre, 2 Prof Id)]
[Prof 1: Enio Romagnome, Prof 2: Roberto Carlos]
[Sala 1: BC-23, Sala 2: BC-08]
